<a href="https://colab.research.google.com/github/kleenman/tds_capstone/blob/pedro/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from fastai.vision.all import *

In [4]:
dataset_path = '/content/gdrive/MyDrive/Capstone/Data/munich_data'

def mask_function(name):
  mask_path = os.path.join(dataset_path, 'masks', name.replace('.jpg', '.npy'))
  np.load(mask_path)


test = mask_function('3278_5338.jpg')

plt.imshow(test)


OSError: [Errno 107] Transport endpoint is not connected: '/content/gdrive/MyDrive/Capstone/Data/munich_data/masks/3278_5338.npy'